Merge excel file

In [ ]:
import pandas as pd
import glob
import os
from google.colab import files  # Helper to download files

# 1. Look for CSV files in the current Colab directory
# The default path for uploaded files in Colab is '/content/'
file_pattern = "/content/*.csv"
all_files = glob.glob(file_pattern)

print(f"Found {len(all_files)} files: {all_files}")

df_list = []

# 2. Loop and combine
for filename in all_files:
    # Skip the output file if it already exists to avoid recursion
    if "combined_demo_data.csv" in filename:
        continue

    try:
        df = pd.read_csv(filename)
        df_list.append(df)
    except Exception as e:
        print(f"Error reading {filename}: {e}")

# 3. Concatenate and Save
if df_list:
    combined_csv = pd.concat(df_list, axis=0, ignore_index=True)

    # Save the result
    output_filename = "combined_demo_data.csv"
    combined_csv.to_csv(output_filename, index=False)
    print(f"Success! {output_filename} created.")

    # 4. Trigger a download to your local computer automatically
    files.download(output_filename)
else:
    print("No CSV files found. Please ensure you uploaded them to the Files tab on the left.")

Biometric updates data- cleaning


In [ ]:
import pandas as pd
import numpy as np
# Load the dataset
df_bio= pd.read_csv('/content/drive/MyDrive/combined_data.csv')
# Inspect the dataset information
print("\nDataset Info:")
print(df_bio.info())

# Check for missing values
print("\nMissing Values:")
print(df_bio.isnull().sum())

# Check for duplicates
print("\nDuplicate Rows:")
print(df_bio.duplicated().sum())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1861108 entries, 0 to 1861107
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   date          object
 1   state         object
 2   district      object
 3   pincode       int64 
 4   bio_age_5_17  int64 
 5   bio_age_17_   int64 
dtypes: int64(3), object(3)
memory usage: 85.2+ MB
None

Missing Values:
date            0
state           0
district        0
pincode         0
bio_age_5_17    0
bio_age_17_     0
dtype: int64

Duplicate Rows:
94896


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Making names of sates uniform throughout the dataset

In [ ]:
# 1. Standardize Column Names
df_bio.rename(columns={'demo_age_17_': 'demo_age_above_17'}, inplace=True)

# 2. Convert Date Column to datetime objects
df_bio['date'] = pd.to_datetime(df_bio['date'], format='%d-%m-%Y', errors='coerce')

# 3. Initial String Standardization (Title Case & Trim Whitespace)
df_bio['state'] = df_bio['state'].astype(str).str.strip().str.title()
df_bio['district'] = df_bio['district'].astype(str).str.strip().str.title()

# 4. Clean State Names
dnhdd_target = 'Dadra and Nagar Haveli and Daman and Diu'
state_mapping = {
    'Orissa': 'Odisha',
    'Pondicherry': 'Puducherry',
    'Jammu & Kashmir': 'Jammu And Kashmir',
    'Jammu and Kashmir': 'Jammu And Kashmir',
    'West Bengli': 'West Bengal',
    'West  Bengal': 'West Bengal',
    'West Bangal': 'West Bengal',
    'Westbengal': 'West Bengal',
    'West bengal': 'West Bengal',
    'Chhatisgarh': 'Chhattisgarh',
    'Andaman & Nicobar Islands': 'Andaman And Nicobar Islands',
    'Andaman and Nicobar Islands': 'Andaman And Nicobar Islands',
    'Dadra And Nagar Haveli': dnhdd_target,
    'Dadra & Nagar Haveli': dnhdd_target,
    'Dadra and Nagar Haveli': dnhdd_target,
    'Daman And Diu': dnhdd_target,
    'Daman and Diu': dnhdd_target,
    'Dadra And Nagar Haveli And Daman And Diu': dnhdd_target,
    'Uttaranchal': 'Uttarakhand',
    # Fixing city names found in state column
    'Jaipur': 'Rajasthan',
    'Nagpur': 'Maharashtra',
    'Darbhanga': 'Bihar',
    'Daman & Diu': dnhdd_target,
    'Tamilnadu': 'Tamil Nadu',
    'Madanapalle': 'Andhra Pradesh',
    'Puttenahalli': 'Karnataka',
    'Raja Annamalai Puram': 'Tamil Nadu',
    'Balanagar': 'Telangana',
    '100000': np.nan  # Mark invalid entries as NaN
}

# Apply the state mapping
df_bio['state'] = df_bio['state'].replace(state_mapping)

# Drop rows with invalid states (where state became NaN)
df_bio = df_bio.dropna(subset=['state'])

# 5. Clean District Names
district_mapping = {
    'Hasan': 'Hassan'
}
df_bio['district'] = df_bio['district'].replace(district_mapping)


# Display info to verify cleaning
print(df_bio.info())
print("\nUnique States:", df_bio['state'].unique())
unique_state_count = df_bio['state'].nunique()
print(f"Unique States Count: {unique_state_count}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1861108 entries, 0 to 1861107
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   state         object        
 2   district      object        
 3   pincode       int64         
 4   bio_age_5_17  int64         
 5   bio_age_17_   int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 85.2+ MB
None

Unique States: ['Haryana' 'Himachal Pradesh' 'Jammu And Kashmir' 'Jharkhand' 'Karnataka'
 'Kerala' 'Lakshadweep' 'Madhya Pradesh' 'Maharashtra'
 'Andaman And Nicobar Islands' 'Andhra Pradesh' 'Arunachal Pradesh'
 'Assam' 'Bihar' 'Uttar Pradesh' 'Uttarakhand' 'West Bengal' 'Chandigarh'
 'Chhattisgarh' 'Dadra and Nagar Haveli and Daman and Diu' 'Delhi' 'Goa'
 'Gujarat' 'Manipur' 'Meghalaya' 'Mizoram' 'Nagaland' 'Odisha'
 'Puducherry' 'Punjab' 'Rajasthan' 'Sikkim' 'Tamil Nadu' 'Telangana'
 'Tripura' 'Ladakh']
Unique States Count: 36


In [ ]:
print("\nChecking for negative values in numerical columns:")

# Columns to check for negative values
numerical_cols = ['pincode', 'bio_age_5_17', 'bio_age_17_']

found_negative = False
for col in numerical_cols:
    if (df_bio[col] < 0).any():
        print(f"  - Column '{col}' contains negative values. Minimum value: {df_bio[col].min()}")
        found_negative = True
    else:
        print(f"  - Column '{col}' does not contain negative values.")

if not found_negative:
    print("No negative values found in the specified numerical columns.")


Checking for negative values in numerical columns:
  - Column 'pincode' does not contain negative values.
  - Column 'bio_age_5_17' does not contain negative values.
  - Column 'bio_age_17_' does not contain negative values.
No negative values found in the specified numerical columns.


In [ ]:
# The dataset is now available in df_bio from the previous step.
# We will use df_bio for inspection.
# Inspect the dataset information
print("\nDataset Info:")
print(df_bio.info())

# Check for missing values
print("\nMissing Values:")
print(df_bio.isnull().sum())

# Check for duplicates
print("\nDuplicate Rows:")
print(df_bio.duplicated().sum())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1861108 entries, 0 to 1861107
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   state         object        
 2   district      object        
 3   pincode       int64         
 4   bio_age_5_17  int64         
 5   bio_age_17_   int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 85.2+ MB
None

Missing Values:
date            0
state           0
district        0
pincode         0
bio_age_5_17    0
bio_age_17_     0
dtype: int64

Duplicate Rows:
95597


In [ ]:
print("\nDropping duplicate rows...")
df_bio.drop_duplicates(inplace=True)

print("\nDuplicate Rows after dropping duplicates:")
print(df_bio.duplicated().sum())


Dropping duplicate rows...

Duplicate Rows after dropping duplicates:
0


Now that the missing values are handled, let's also check the `Dataset Info` and `Duplicate Rows` again to ensure the data is clean after dropping rows.

In [ ]:
print("\nDataset Info after dropping NaN rows:")
print(df_bio.info())

print("\nDuplicate Rows after dropping NaN rows:")
print(df_bio.duplicated().sum())


Dataset Info after dropping NaN rows:
<class 'pandas.core.frame.DataFrame'>
Index: 1765511 entries, 0 to 1861107
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   state         object        
 2   district      object        
 3   pincode       int64         
 4   bio_age_5_17  int64         
 5   bio_age_17_   int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 94.3+ MB
None

Duplicate Rows after dropping NaN rows:
0


Demographoc


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/combined_demo_data.csv')
# Inspect the dataset information
print("\nDataset Info:")
print(df.info())

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Check for duplicates
print("\nDuplicate Rows:")
print(df.duplicated().sum())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2071700 entries, 0 to 2071699
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   date           object
 1   state          object
 2   district       object
 3   pincode        int64 
 4   demo_age_5_17  int64 
 5   demo_age_17_   int64 
dtypes: int64(3), object(3)
memory usage: 94.8+ MB
None

Missing Values:
date             0
state            0
district         0
pincode          0
demo_age_5_17    0
demo_age_17_     0
dtype: int64

Duplicate Rows:
473601


In [ ]:
import pandas as pd

# Load the dataset
df_demo = pd.read_csv('/content/drive/MyDrive/combined_demo_data.csv')

# 1. Clean whitespace in state names (removes leading/trailing spaces)
df_demo['state'] = df_demo['state'].str.strip()

# 2. Define a dictionary to map incorrect names to correct state names
state_corrections = {
    # Variations of Odisha
    'Orissa': 'Odisha',
    'ODISHA': 'Odisha',
    'odisha': 'Odisha',

    # Variations of West Bengal
    'West Bengli': 'West Bengal',
    'West  Bengal': 'West Bengal',
    'West Bangal': 'West Bengal',
    'Westbengal': 'West Bengal',
    'West bengal': 'West Bengal',
    'WESTBENGAL': 'West Bengal',
    'WEST BENGAL': 'West Bengal',
    'west Bengal': 'West Bengal',

    # Variations of UTs and other states
    'Jammu & Kashmir': 'Jammu and Kashmir',
    'Andaman & Nicobar Islands': 'Andaman and Nicobar Islands',
    'Uttaranchal': 'Uttarakhand',
    'Pondicherry': 'Puducherry',
    'Chhatisgarh': 'Chhattisgarh',
    'Dadra & Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'Dadra and Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'Daman & Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Daman and Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'andhra pradesh': 'Andhra Pradesh',

    # Fix Cities/Districts appearing as States based on context
    'BALANAGAR': 'Telangana',
    'Jaipur': 'Rajasthan',
    'Madanapalle': 'Andhra Pradesh',
    'Nagpur': 'Maharashtra',
    'Raja Annamalai Puram': 'Tamil Nadu',
    'Darbhanga': 'Bihar',
    'Puttenahalli': 'Karnataka'
}

# 3. Apply the corrections
df_demo['state'] = df_demo['state'].replace(state_corrections)

# 4. Remove invalid rows where state is '100000'
df_demo = df_demo[df_demo['state'] != '100000']

print("Cleaning complete. Unique states:")
print(df_demo['state'].unique())

print("Final List of States & UTs:")
print(df_demo['state'].unique())

print("\nFinal Count of States & UTs:")
print(len(df_demo['state'].unique()))

Cleaning complete. Unique states:
['Karnataka' 'Kerala' 'Ladakh' 'Madhya Pradesh' 'Maharashtra' 'Punjab'
 'Rajasthan' 'Sikkim' 'Tamil Nadu' 'Telangana' 'Manipur' 'Meghalaya'
 'Mizoram' 'Nagaland' 'Odisha' 'Puducherry' 'Jammu and Kashmir'
 'Jharkhand' 'Uttar Pradesh' 'Uttarakhand' 'West Bengal'
 'Andaman and Nicobar Islands' 'Andhra Pradesh' 'Assam' 'Bihar'
 'Chandigarh' 'Chhattisgarh' 'Dadra and Nagar Haveli and Daman and Diu'
 'Delhi' 'Goa' 'Gujarat' 'Haryana' 'Himachal Pradesh' 'Arunachal Pradesh'
 'Tripura' 'Lakshadweep']
Final List of States & UTs:
['Karnataka' 'Kerala' 'Ladakh' 'Madhya Pradesh' 'Maharashtra' 'Punjab'
 'Rajasthan' 'Sikkim' 'Tamil Nadu' 'Telangana' 'Manipur' 'Meghalaya'
 'Mizoram' 'Nagaland' 'Odisha' 'Puducherry' 'Jammu and Kashmir'
 'Jharkhand' 'Uttar Pradesh' 'Uttarakhand' 'West Bengal'
 'Andaman and Nicobar Islands' 'Andhra Pradesh' 'Assam' 'Bihar'
 'Chandigarh' 'Chhattisgarh' 'Dadra and Nagar Haveli and Daman and Diu'
 'Delhi' 'Goa' 'Gujarat' 'Haryana' 'Himachal

In [ ]:
print("\nChecking for negative values in numerical columns:")

# Columns to check for negative values
numerical_cols = ['pincode', 'demo_age_5_17', 'demo_age_17_']

found_negative = False
for col in numerical_cols:
    if (df_demo[col] < 0).any():
        print(f"  - Column '{col}' contains negative values. Minimum value: {df_demo[col].min()}")
        found_negative = True
    else:
        print(f"  - Column '{col}' does not contain negative values.")

if not found_negative:
    print("No negative values found in the specified numerical columns.")


Checking for negative values in numerical columns:
  - Column 'pincode' does not contain negative values.
  - Column 'demo_age_5_17' does not contain negative values.
  - Column 'demo_age_17_' does not contain negative values.
No negative values found in the specified numerical columns.


In [ ]:
#After cleaning
print(df_demo.info())

# Check for missing values
print("\nMissing Values:")
print(df_demo.isnull().sum())

# Check for duplicates
print("\nDuplicate Rows:")
print(df_demo.duplicated().sum())


<class 'pandas.core.frame.DataFrame'>
Index: 2071698 entries, 0 to 2071699
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   date           object
 1   state          object
 2   district       object
 3   pincode        int64 
 4   demo_age_5_17  int64 
 5   demo_age_17_   int64 
dtypes: int64(3), object(3)
memory usage: 110.6+ MB
None

Missing Values:
date             0
state            0
district         0
pincode          0
demo_age_5_17    0
demo_age_17_     0
dtype: int64

Duplicate Rows:
474305


In [ ]:
# Find all rows involved in duplication
# keep=False marks all identical rows as True
duplicates = df_demo[df_demo.duplicated(keep=False)]

# Sort to group identical rows
duplicates_sorted = duplicates.sort_values(by=list(df_demo.columns))

# Display the first 20 rows
print(duplicates_sorted.head(20).to_markdown(index=False))

# Calculate counts
num_redundant = df_demo.duplicated().sum()
print(f"Number of redundant rows to be removed: {num_redundant}")

| date       | state                       | district                 |   pincode |   demo_age_5_17 |   demo_age_17_ |
|:-----------|:----------------------------|:-------------------------|----------:|----------------:|---------------:|
| 01-03-2025 | Andaman and Nicobar Islands | Nicobar                  |    744301 |              16 |            180 |
| 01-03-2025 | Andaman and Nicobar Islands | Nicobar                  |    744301 |              16 |            180 |
| 01-03-2025 | Andaman and Nicobar Islands | North And Middle Andaman |    744202 |              10 |            201 |
| 01-03-2025 | Andaman and Nicobar Islands | North And Middle Andaman |    744202 |              10 |            201 |
| 01-03-2025 | Andhra Pradesh              | Adilabad                 |    504105 |              17 |             61 |
| 01-03-2025 | Andhra Pradesh              | Adilabad                 |    504105 |              17 |             61 |
| 01-03-2025 | Andhra Pradesh              | Adi

In [ ]:
# Remove duplicates, keeping the first occurrence
# This keeps exactly one entry for each set of duplicate rows
df_demo_deduplicated = df_demo.drop_duplicates(keep='first')

# Output results
print(f"Duplicates removed: {len(df_demo) - len(df_demo_deduplicated)}")
print(f"Final row count: {len(df_demo_deduplicated)}")
df_demo=df_demo_deduplicated
df_demo_deduplicated.info()
print("\nDuplicate Rows:")
print(df_demo.duplicated().sum())


Duplicates removed: 474305
Final row count: 1597393
<class 'pandas.core.frame.DataFrame'>
Index: 1597393 entries, 0 to 2071699
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   date           1597393 non-null  object
 1   state          1597393 non-null  object
 2   district       1597393 non-null  object
 3   pincode        1597393 non-null  int64 
 4   demo_age_5_17  1597393 non-null  int64 
 5   demo_age_17_   1597393 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 85.3+ MB

Duplicate Rows:
0


Enrollement


In [ ]:
import pandas as pd

# Load the dataset
df_enroll = pd.read_csv('/content/drive/MyDrive/combined_enroll_data.csv')
# Inspect the dataset information
print("\nDataset Info:")
print(df_enroll.info())

# Check for missing values
print("\nMissing Values:")
print(df_enroll.isnull().sum())

# Check for duplicates
print("\nDuplicate Rows:")
print(df_enroll.duplicated().sum())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006029 entries, 0 to 1006028
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   date            1006029 non-null  object
 1   state           1006029 non-null  object
 2   district        1006029 non-null  object
 3   pincode         1006029 non-null  int64 
 4   age_0_5         1006029 non-null  int64 
 5   age_5_17        1006029 non-null  int64 
 6   age_18_greater  1006029 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 53.7+ MB
None

Missing Values:
date              0
state             0
district          0
pincode           0
age_0_5           0
age_5_17          0
age_18_greater    0
dtype: int64

Duplicate Rows:
22957


In [ ]:
import pandas as pd

# Load the dataframe
df_enroll= pd.read_csv('/content/drive/MyDrive/combined_enroll_data.csv')

# Define the mapping
state_corrections_enroll = {
    'Orissa': 'Odisha',
    'ODISHA': 'Odisha',
    'Pondicherry': 'Puducherry',
    'The Dadra And Nagar Haveli And Daman And Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Dadra & Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'Dadra and Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'Daman & Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Daman and Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Andaman & Nicobar Islands': 'Andaman and Nicobar Islands',
    'WEST BENGAL': 'West Bengal',
    'West  Bengal': 'West Bengal',
    'West Bangal': 'West Bengal',
    'Westbengal': 'West Bengal',
    'West bengal': 'West Bengal',
    'WESTBENGAL': 'West Bengal',
    'Jammu And Kashmir': 'Jammu and Kashmir',
    'Jammu & Kashmir': 'Jammu and Kashmir',
    'andhra pradesh': 'Andhra Pradesh'
}

# Clean whitespace
df_enroll['state'] = df_enroll['state'].str.strip()

# Apply corrections
df_enroll['state'] = df_enroll['state'].replace(state_corrections_enroll)

# Remove '100000'
df_enroll = df_enroll[df_enroll['state'] != '100000']

print("Final List of States & UTs:")
print(df_enroll['state'].unique())

print("\nFinal Count of States & UTs:")
print(len(df_enroll['state'].unique()))

Final List of States & UTs:
['Karnataka' 'Kerala' 'Ladakh' 'Lakshadweep' 'Madhya Pradesh'
 'Maharashtra' 'Manipur' 'Meghalaya' 'Mizoram' 'Nagaland' 'Odisha'
 'Puducherry' 'Punjab' 'Rajasthan' 'Sikkim' 'Tamil Nadu' 'Telangana'
 'Tripura' 'Uttar Pradesh' 'Uttarakhand' 'West Bengal' 'Andhra Pradesh'
 'Arunachal Pradesh' 'Assam' 'Bihar' 'Chandigarh' 'Chhattisgarh'
 'Dadra and Nagar Haveli and Daman and Diu' 'Delhi' 'Goa' 'Gujarat'
 'Haryana' 'Himachal Pradesh' 'Jammu and Kashmir' 'Jharkhand'
 'Andaman and Nicobar Islands']

Final Count of States & UTs:
36


In [ ]:
# Find all rows involved in duplication
# keep=False marks all identical rows as True
duplicates = df_enroll[df_enroll.duplicated(keep=False)]

# Sort to group identical rows
duplicates_sorted = duplicates.sort_values(by=list(df_enroll.columns))

# Display the first 20 rows
print(duplicates_sorted.head(20).to_markdown(index=False))

# Calculate counts
num_redundant = df_enroll.duplicated().sum()
print(f"Number of redundant rows to be removed: {num_redundant}")

| date       | state                       | district      |   pincode |   age_0_5 |   age_5_17 |   age_18_greater |
|:-----------|:----------------------------|:--------------|----------:|----------:|-----------:|-----------------:|
| 01-09-2025 | Andaman and Nicobar Islands | South Andaman |    744103 |         1 |          0 |                0 |
| 01-09-2025 | Andaman and Nicobar Islands | South Andaman |    744103 |         1 |          0 |                0 |
| 01-09-2025 | Andaman and Nicobar Islands | South Andaman |    744207 |         1 |          0 |                0 |
| 01-09-2025 | Andaman and Nicobar Islands | South Andaman |    744207 |         1 |          0 |                0 |
| 01-09-2025 | Odisha                      | Balangir      |    767066 |         2 |          0 |                0 |
| 01-09-2025 | Odisha                      | Balangir      |    767066 |         2 |          0 |                0 |
| 01-09-2025 | Odisha                      | Baleshwar     |    

In [ ]:
# Remove duplicates, keeping the first occurrence
# This keeps exactly one entry for each set of duplicate rows
df_enroll_deduplicated = df_enroll.drop_duplicates(keep='first')

# Output results
print(f"Duplicates removed: {len(df_enroll) - len(df_enroll_deduplicated)}")
print(f"Final row count: {len(df_enroll_deduplicated)}")
df_enroll=df_enroll_deduplicated
df_enroll_deduplicated.info()
print("\nDuplicate Rows:")
print(df_enroll.duplicated().sum())


Duplicates removed: 23432
Final row count: 982575
<class 'pandas.core.frame.DataFrame'>
Index: 982575 entries, 0 to 1004911
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   date            982575 non-null  object
 1   state           982575 non-null  object
 2   district        982575 non-null  object
 3   pincode         982575 non-null  int64 
 4   age_0_5         982575 non-null  int64 
 5   age_5_17        982575 non-null  int64 
 6   age_18_greater  982575 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 60.0+ MB

Duplicate Rows:
0


In [ ]:
print("\nChecking for negative values in numerical columns:")

# Columns to check for negative values
numerical_cols = ['pincode', 'age_0_5', 'age_5_17','age_18_greater']

found_negative = False
for col in numerical_cols:
    if (df_enroll[col] < 0).any():
        print(f"  - Column '{col}' contains negative values. Minimum value: {df_enroll[col].min()}")
        found_negative = True
    else:
        print(f"  - Column '{col}' does not contain negative values.")

if not found_negative:
    print("No negative values found in the specified numerical columns.")


Checking for negative values in numerical columns:
  - Column 'pincode' does not contain negative values.
  - Column 'age_0_5' does not contain negative values.
  - Column 'age_5_17' does not contain negative values.
  - Column 'age_18_greater' does not contain negative values.
No negative values found in the specified numerical columns.


In [ ]:

print("\nDataset Info:")
print(df_enroll.info())

# Check for missing values
print("\nMissing Values:")
print(df_enroll.isnull().sum())

# Check for duplicates
print("\nDuplicate Rows:")
print(df_enroll.duplicated().sum())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 982575 entries, 0 to 1004911
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   date            982575 non-null  object
 1   state           982575 non-null  object
 2   district        982575 non-null  object
 3   pincode         982575 non-null  int64 
 4   age_0_5         982575 non-null  int64 
 5   age_5_17        982575 non-null  int64 
 6   age_18_greater  982575 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 60.0+ MB
None

Missing Values:
date              0
state             0
district          0
pincode           0
age_0_5           0
age_5_17          0
age_18_greater    0
dtype: int64

Duplicate Rows:
0


In [ ]:
import altair as alt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Clean District Names
df_bio['district'] = df_bio['district'].str.strip()
df_enroll['district'] = df_enroll['district'].str.strip()

# --- PART A: PREPARING THE MASTER DATASET ---

# Calculate total_enrolment in df_enroll
df_enroll['total_enrolment'] = df_enroll['age_0_5'] + df_enroll['age_5_17'] + df_enroll['age_18_greater']

bio_grouped = df_bio.groupby(['state', 'district'])['bio_age_5_17'].sum().reset_index()
bio_grouped.rename(columns={'bio_age_5_17': 'total_updates'}, inplace=True)

enrol_grouped = df_enroll.groupby(['state', 'district'])['total_enrolment'].sum().reset_index()

df_master = pd.merge(enrol_grouped, bio_grouped, on=['state', 'district'], how='inner')

median_enrol = df_master['total_enrolment'].median()
df_master = df_master[df_master['total_enrolment'] > median_enrol].copy()

df_master['update_efficiency'] = df_master['total_updates'] / df_master['total_enrolment']
df_master['log_enrolment'] = np.log1p(df_master['total_enrolment'])

# --- PART B: K-MEANS CLUSTERING ---
X = df_master[['log_enrolment', 'update_efficiency']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_master['cluster'] = kmeans.fit_predict(X_scaled)

# --- PART C: CLUSTER NAMING LOGIC ---
centroids = scaler.inverse_transform(kmeans.cluster_centers_)
centroid_df = pd.DataFrame(centroids, columns=['log_enrolment', 'update_efficiency'])
centroid_df['cluster'] = range(4)
centroid_df = centroid_df.sort_values('log_enrolment', ascending=False)

high_vol_indices = centroid_df.head(2).index
low_vol_indices = centroid_df.tail(2).index

high_vol_clusters = centroid_df.loc[high_vol_indices].sort_values('update_efficiency', ascending=False)
mature_cluster = high_vol_clusters.iloc[0]['cluster']
stress_cluster = high_vol_clusters.iloc[1]['cluster']

low_vol_clusters = centroid_df.loc[low_vol_indices].sort_values('update_efficiency', ascending=False)
dormant_cluster = low_vol_clusters.iloc[0]['cluster']
laggard_cluster = low_vol_clusters.iloc[1]['cluster']

persona_map = {
    mature_cluster: 'Mature/Stable',
    stress_cluster: 'High-Growth Stress',
    dormant_cluster: 'Dormant',
    laggard_cluster: 'Critical Laggards'
}
df_master['Persona'] = df_master['cluster'].map(persona_map)

# --- PART D: INTERACTIVE ALTAIR CHART ---
chart = alt.Chart(df_master).mark_circle(size=60).encode(
    x=alt.X('total_enrolment', scale=alt.Scale(type='log'), title='Total Enrollment (Log Scale)') ,
    y=alt.Y('update_efficiency', title='Update Efficiency'),
    color=alt.Color('Persona', scale=alt.Scale(scheme='set1'), legend=alt.Legend(title="District Persona")),
    tooltip=['district', 'state', 'total_enrolment', 'total_updates', 'update_efficiency', 'Persona']
).properties(
    title='Interactive District Vulnerability Matrix',
    width=700,
    height=500
).interactive()

chart.show()

alt.Chart(...)

In [ ]:
df_enroll['district'] = df_enroll['district'].str.strip()

# 2. Compute "Pincode-Level" Activity
# We sum all enrollments for each pincode to see the total service delivered to that locality
pincode_totals = df_enroll.groupby(['state', 'district', 'pincode'])['total_enrolment'].sum().reset_index()

# 3. Calculate Inequality Metrics per District
# We use the Coefficient of Variation (CV) = Std Dev / Mean
district_stats = pincode_totals.groupby(['state', 'district'])['total_enrolment'].agg(['std', 'mean', 'sum']).reset_index()

# Filter for significant districts (ignore tiny ones where data is noisy)
median_activity = district_stats['sum'].median()
significant_districts = district_stats[district_stats['sum'] > median_activity].copy()

# Formula: Inequality Index
# We add +1 to mean to avoid division by zero errors for dormant districts
significant_districts['inequality_index'] = significant_districts['std'] / (significant_districts['mean'] + 1)

# 4. Select Top Offenders (Highest Inequality)
top_unequal = significant_districts.sort_values(by='inequality_index', ascending=False).head(20)

# 5. Create Interactive Chart
chart = alt.Chart(top_unequal).mark_bar().encode(
    x=alt.X('district', sort='-y', title='District'),
    y=alt.Y('inequality_index', title='Inequality Index (Coefficient of Variation)'),
    color=alt.Color('state', legend=alt.Legend(title="State")),
    tooltip=['district', 'state', 'inequality_index', 'sum']
).properties(
    title='Top 20 Districts with Highest Service Inequality (Hidden Pockets of Exclusion)',
    width=800,
    height=500
).interactive()
chart.show()

alt.Chart(...)